<a href="https://colab.research.google.com/github/RCSnyder/Sales_Forecasting/blob/kishen-model/5214_Clean_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
#Read in .csv files into pandas dataframes
train_lag   = pd.read_csv('/content/drive/My Drive/CSCE 5214 Group #1 /new_month_group.csv')
items       = pd.read_csv('/content/drive/My Drive/CSCE 5214 Group #1 /items.csv')

In [45]:
#Down casts the data entries from int64 to int32 and float64 to float32
#This reduces the size of the records by almost half. (From 134mb to 61mb)
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [46]:
train_lag  = downcast_dtypes(train_lag)
items      = downcast_dtypes(items)

In [52]:
train_lag

,month,shop_id,item_category_id,item_id,item_price,mon_lag_1,mon_lag_2,mon_lag_3,mon_lag_4,mon_lag_5,item_cnt_day
316176,june,2,2,5572,1490.0,2.0,2.0,1.0,1.0,4.0,2.0
316177,june,2,2,5573,499.0,0.0,0.0,0.0,2.0,1.0,2.0
316178,june,2,2,5574,499.0,0.0,2.0,2.0,3.0,0.0,1.0
316179,june,2,2,5575,1090.0,0.0,3.0,1.0,5.0,1.0,1.0
316180,june,2,2,5625,2490.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1739017,october,59,79,17717,1250.0,1.0,1.0,4.0,2.0,1.0,4.0
1739018,october,59,79,17717,1999.0,1.0,1.0,4.0,2.0,1.0,1.0
1739019,october,59,83,22087,119.0,3.0,2.0,5.0,1.0,2.0,6.0
1739020,october,59,83,22088,119.0,1.0,7.0,7.0,4.0,3.0,2.0


In [51]:
train_lag = train_lag.dropna()
train_lag.drop(labels=['date_block_num'], axis=1, inplace = True)

In [53]:
# stores all feature columns in the x variable, and stores the target variable as the last column in y variable
x = train_lag.iloc[:, :-1].values
y = train_lag.iloc[:,-1].values

In [54]:
# Encoding categorical data
# provides a value to data that can be then used in regression equations, eg. Friday = 1 etc.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer([('encoder', OneHotEncoder(), [0, 1, 2, 3])], remainder = 'passthrough')
x = ct.fit_transform(x)

In [55]:
# split data set into training set and testing set
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [56]:
# fitting multiple linear regression to the training set
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, Y_train)

# prediction of test split from original data, train_grouped_month
y_prediction = regressor.predict(X_test)

# The output of the this is the r squared
regressor.score(X_test, Y_test)

0.4795410598431261

In [58]:
#AUTOML CODE
#Download 64bit java
!pip install h2o
import h2o
from h2o.automl import H2OAutoML

h2o.init()
train_f = h2o.H2OFrame(train_lag)

splits = train_f.split_frame(ratios = [0.8], seed = 1)
train_split = splits[0]
test_split = splits[1]

aml = H2OAutoML(max_runtime_secs = 1000, seed = 1, project_name = "sales_forecasting")
aml.train(y = "item_cnt_day", training_frame = train_split, leaderboard_frame = test_split)
lb = aml.leaderboard

     |████████████████████████████████| 129.4MB 86kB/s 
  Created wheel for h2o: filename=h2o-3.30.1.2-py2.py3-none-any.whl size=129446949 sha256=4c5d8edfb3e7c400ec35b992e766de740aa808bf0f343b321711b13d83c63f12
  Stored in directory: /root/.cache/pip/wheels/c6/be/83/a33a3c1c97fce1d136222bf4ed6d79da405ef6103f5b434c1e
Successfully built h2o
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.8" 2020-07-14; OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphk125fr5
  JVM stdout: /tmp/tmphk125fr5/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmphk125fr5/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
C

H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_ld8kiy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%


In [59]:
#shows leaderboard of model competition
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200922_185541,24.9582,4.99581,24.9582,1.16417,nan
XGBoost_grid__1_AutoML_20200922_185541_model_1,25.6044,5.06008,25.6044,1.20894,nan
StackedEnsemble_BestOfFamily_AutoML_20200922_185541,25.6464,5.06422,25.6464,1.17222,nan
GBM_grid__1_AutoML_20200922_185541_model_1,31.612,5.62245,31.612,1.31088,nan
GLM_1_AutoML_20200922_185541,32.3694,5.68941,32.3694,1.26201,nan
DeepLearning_grid__1_AutoML_20200922_185541_model_1,32.5609,5.70621,32.5609,1.21225,nan
DeepLearning_grid__2_AutoML_20200922_185541_model_1,34.1782,5.84621,34.1782,1.30448,nan
DeepLearning_grid__3_AutoML_20200922_185541_model_1,35.9945,5.99954,35.9945,1.34664,nan
DeepLearning_1_AutoML_20200922_185541,41.2221,6.42044,41.2221,1.39217,nan
XRT_1_AutoML_20200922_185541,41.8336,6.46789,41.8336,1.24035,nan


In [60]:
prediction = aml.leader.model_performance(test_split)
prediction


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 24.95816617923115
RMSE: 4.9958148663887805
MAE: 1.1641668040412985
RMSLE: NaN
R^2: 0.5971352594080147
Mean Residual Deviance: 24.95816617923115
Null degrees of freedom: 284443
Residual degrees of freedom: 284438
Null deviance: 17621797.54175254
Residual deviance: 7099200.620685225
AIC: 1722344.6455519388


In [61]:
def display_scores(scores):
    print("rmse Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [62]:
# Cross validation
# Approx 136 000 data points per fold
from sklearn.model_selection import cross_val_score as cvs
reg_score = cvs(regressor, X_train, Y_train, scoring = "neg_mean_squared_error", cv = 10)
reg_score = np.sqrt(-reg_score)

array([6.23001342, 5.45725128, 5.44668999, 6.49458659, 6.28875826,
       6.10155977, 6.47765804, 6.58502495, 9.35579312, 5.13810962])

In [64]:
display_scores(reg_score)

rmse Scores: [6.23001342 5.45725128 5.44668999 6.49458659 6.28875826 6.10155977
 6.47765804 6.58502495 9.35579312 5.13810962]
Mean: 6.357544504285576
Standard deviation: 1.1085205949939727


In [65]:
# Ridge regression

from sklearn.linear_model import Ridge
ridge_reg = Ridge()
ridge_reg.fit(X_train, Y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [67]:
# Cross validation for Ridge

ridge_score = cvs(ridge_reg, X_train, Y_train, scoring = "neg_mean_squared_error", cv = 10)
ridge_score = np.sqrt(-ridge_score)
display_scores(ridge_score)

rmse Scores: [6.21875968 5.62301369 5.51276386 6.60417573 6.28425327 6.18583367
 6.15746878 6.86119511 9.3257777  5.09765849]
Mean: 6.387089997129627
Standard deviation: 1.0977622784354573
